<a href="https://colab.research.google.com/github/ParalelaUCM/biciMAD/blob/master/Practica_Spark_Bicimad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
#Setup
Instalamos y configuramos las herramientas necesarias para empezar a trabajar


In [0]:
!apt-get install openjdk-8-jdk
!apt install unzip
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!pip install pyspark

In [0]:
!apt install unzip

---

#**Descargamos los datasets de bicibad**

Descargaremos cada dataset de la [web oficial](https://opendata.emtmadrid.es/Datos-estaticos/Datos-generales-(1)) de **biciMAD**. Los renombraremos a '*.zip*' para poder descomprimirlos y una vez descomprimidos los movemos a la carpeta '*datasets*'. Por último eliminamos los ficheros comprimidos.

In [0]:
#Carpeta para el dataset
!mkdir dataset

In [4]:
#Enero de 2019
!wget -N "https://opendata.emtmadrid.es/getattachment/2ebcc70a-4914-43c6-9ada-a0f0520032a4/201901_Usage_Bicimad.aspx"
!mv 201901_Usage_Bicimad.aspx 201901_Usage_Bicimad.zip
!unzip 201901_Usage_Bicimad.zip
!mv 201901_Usage_Bicimad.json dataset
!rm 201901_Usage_Bicimad.zip

--2020-05-24 16:21:31--  https://opendata.emtmadrid.es/getattachment/2ebcc70a-4914-43c6-9ada-a0f0520032a4/201901_Usage_Bicimad.aspx
Resolving opendata.emtmadrid.es (opendata.emtmadrid.es)... 185.79.130.130
Connecting to opendata.emtmadrid.es (opendata.emtmadrid.es)|185.79.130.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78311972 (75M) [application/zip]
Saving to: ‘201901_Usage_Bicimad.aspx’

201901_Usage_Bicima 100%[===================>]  74.68M  1.85MB/s    in 45s     

2020-05-24 16:22:16 (1.67 MB/s) - ‘201901_Usage_Bicimad.aspx’ saved [78311972/78311972]

Archive:  201901_Usage_Bicimad.zip
  inflating: 201901_Usage_Bicimad.json  


In [0]:
'''
#Febrero de 2019
!wget -N "https://opendata.emtmadrid.es/getattachment/aa8c34d2-ddba-46d9-b6db-882c0b4a12f0/201902_Usage_Bicimad.aspx"
!mv 201902_Usage_Bicimad.aspx 201902_Usage_Bicimad.zip 
!unzip 201902_Usage_Bicimad.zip
!mv 201902_Usage_Bicimad.json dataset
!rm 201902_Usage_Bicimad.zip
'''

In [0]:
'''
#Junio de 2019
!wget -N "https://opendata.emtmadrid.es/getattachment/7517a650-ccdf-4ab1-b1b0-a1d13694472e/201906_Usage_Bicimad.aspx"
!mv 201906_Usage_Bicimad.aspx 201906_Usage_Bicimad.zip
!unzip 201906_Usage_Bicimad.zip
!mv 201906_Usage_Bicimad.json dataset
!rm 201906_Usage_Bicimad.zip
'''

---
#Ahora ya podemos empezar:

In [0]:
#Aquí ya podemos empezar a trabajar
import json
from pyspark import SparkContext
sc = SparkContext()

In [0]:
rdd_raw  = sc.textFile('/content/dataset/201901_Usage_Bicimad.json')

In [9]:
rdd_raw.take(1)

['{ "_id" : { "$oid" : "5c4b07ea2f38432e007daab8" }, "user_day_code" : "b3b5b61509ab89afbf4d1e589d229bebe6637c2de819f5d612762c939b8d7f1f", "idplug_base" : 14, "track" : { "type" : "FeatureCollection", "features" : [ { "geometry" : { "type" : "Point", "coordinates" : [ -3.6803143, 40.4048615 ] }, "type" : "Feature", "properties" : { "var" : "28007,ES,Madrid,Madrid,AVDA MENENDEZ PELAYO 117,Madrid", "speed" : 1.77, "secondsfromstart" : 153 } }, { "geometry" : { "type" : "Point", "coordinates" : [ -3.67770349972222, 40.4029987997222 ] }, "type" : "Feature", "properties" : { "var" : "28007,ES,Madrid,Madrid,AVDA CIUDAD DE BARCELONA 51,Madrid", "speed" : 6.58, "secondsfromstart" : 93 } }, { "geometry" : { "type" : "Point", "coordinates" : [ -3.67432209972222, 40.4009430997222 ] }, "type" : "Feature", "properties" : { "var" : "28007,ES,Madrid,Madrid,AVDA CIUDAD DE BARCELONA 81,Madrid", "speed" : 4.72, "secondsfromstart" : 32 } } ] }, "user_type" : 1, "idunplug_base" : 23, "travel_time" : 162, 

In [0]:
def mapper(line):
    data = json.loads(line)
    id_usuario = data['_id']['$oid']
    rango_edad = data['ageRange']
    tipo_usuario = data['user_type']
    id_usuario_dia = data['user_day_code']
    start = data['idunplug_station']
    end = data['idplug_station']
    time = data['travel_time']
    dia = data['unplug_hourTime']['$date'][:10]
    hora = data['unplug_hourTime']['$date'][11:12]
    return id_usuario, rango_edad
    '''
    return u_t, u_c, start, end, time
    '''

In [0]:
rdd = rdd_raw.map(mapper)

In [12]:
rdd.take(5)

[('5c4b07ea2f38432e007daab8', 0),
 ('5c4b07ea2f38432e007daaba', 5),
 ('5c4b07ea2f38432e007daabb', 0),
 ('5c4b07ea2f38432e007daabd', 3),
 ('5c4b07ea2f38432e007daabe', 4)]

In [13]:
!pip show py4j

Name: py4j
Version: 0.10.7
Summary: Enables Python programs to dynamically access arbitrary Java objects
Home-page: https://www.py4j.org/
Author: Barthelemy Dagenais
Author-email: barthelemy@infobart.com
License: BSD License
Location: /usr/local/lib/python3.6/dist-packages
Requires: 
Required-by: pyspark
